# CS395 - Assignment 5
### Multivariate LSTMs

By: Joshua Swick  
Date: Feb 20, 2019

## Multiple Input Series

### 1.
Run the given code once and show the resulting output.

In [11]:
# multivariate data preparation
from numpy import array
from numpy import hstack
# multivariate lstm
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [10]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
print(dataset)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


### 2.
Run the above code and show the resulting output.

In [12]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))

# choose a number of time steps
n_steps = 3

# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)

# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(7, 3, 2) (7,)
[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


### 3.
Run the given code 3 times. Show input once and all outputs.

In [23]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))

# choose a number of time steps
n_steps = 3

# convert into input/output
X, y = split_sequences(dataset, n_steps)

# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]

for run in range(3):
    # define model
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(X, y, epochs=200, verbose=0)
    # demonstrate prediction
    x_input = array([[80, 85], [90, 95], [100, 105]])
    x_input = x_input.reshape((1, n_steps, n_features))

    yhat = model.predict(x_input, verbose=0)
    print(f"Multivariate Vanilla LSTM, run {run+1}: {yhat}")

print(f"\nInput:\n{x_input}")

Multivariate Vanilla LSTM, run 1: [[207.51753]]
Multivariate Vanilla LSTM, run 2: [[208.62448]]
Multivariate Vanilla LSTM, run 3: [[214.9445]]

Input:
[[[ 80  85]
  [ 90  95]
  [100 105]]]


### 4.
Change the activation function from 'relu' to 2 others. Run 3 times show code and all outputs.

In [24]:
for run in range(3):
    # define model
    model = Sequential()
    model.add(LSTM(50, activation='tanh', input_shape=(n_steps, n_features)))  # Tanh activation function
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(X, y, epochs=200, verbose=0)
    # demonstrate prediction
    x_input = array([[80, 85], [90, 95], [100, 105]])
    x_input = x_input.reshape((1, n_steps, n_features))

    yhat = model.predict(x_input, verbose=0)
    print(f"Multivariate Vanilla LSTM, TanH activation function, run {run+1}: {yhat}")

Multivariate Vanilla LSTM, TanH activation function, run 1: [[11.400594]]
Multivariate Vanilla LSTM, TanH activation function, run 2: [[10.514781]]
Multivariate Vanilla LSTM, TanH activation function, run 3: [[8.939142]]


In [25]:
for run in range(3):
    # define model
    model = Sequential()
    model.add(LSTM(50, activation='sigmoid', input_shape=(n_steps, n_features)))  # Sigmoid activation function
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(X, y, epochs=200, verbose=0)
    # demonstrate prediction
    x_input = array([[80, 85], [90, 95], [100, 105]])
    x_input = x_input.reshape((1, n_steps, n_features))

    yhat = model.predict(x_input, verbose=0)
    print(f"Multivariate Vanilla LSTM, Sigmoid activation function, run {run+1}: {yhat}")

Multivariate Vanilla LSTM, Sigmoid activation function, run 1: [[6.368427]]
Multivariate Vanilla LSTM, Sigmoid activation function, run 2: [[8.053413]]
Multivariate Vanilla LSTM, Sigmoid activation function, run 3: [[6.3933735]]


## Multiple Parallel Series

### 5.
Run the given code, show the code and resulting output.

In [28]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [29]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))

# choose a number of time steps
n_steps = 3

# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)

# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(6, 3, 3) (6, 3)
[[10 15 25]
 [20 25 45]
 [30 35 65]] [40 45 85]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [ 50  55 105]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [ 60  65 125]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [ 70  75 145]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [ 80  85 165]
[[ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]] [ 90  95 185]


### 6.
Run the given code 3 times. Show input and all outputs.

In [34]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))

# choose a number of time steps
n_steps = 3

# convert into input/output
X, y = split_sequences(dataset, n_steps)

# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]

for run in range(3):
    # define model
    model = Sequential()
    model.add(LSTM(100, activation='relu', return_sequences=True,
    input_shape=(n_steps, n_features)))
    model.add(LSTM(100, activation='relu'))
    model.add(Dense(n_features))
    model.compile(optimizer='adam', loss='mse')

    # fit model
    model.fit(X, y, epochs=400, verbose=0)

    # demonstrate prediction
    x_input = array([[70,75,145], [80,85,165], [90,95,185]])
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)

    print(f"Multivariate Stacked LSTM, run {run+1}: {yhat}")
    
print(f"\nInput:\n{x_input}")

Multivariate Stacked LSTM, run 1: [[101.29086 106.05054 208.67036]]
Multivariate Stacked LSTM, run 2: [[101.12802 107.11191 208.00426]]
Multivariate Stacked LSTM, run 3: [[100.28194 104.88491 204.3823 ]]

Input:
[[[ 70  75 145]
  [ 80  85 165]
  [ 90  95 185]]]


### 7.
Change the activation function from 'relu' to 2 others. Run each 3 times show code and all outputs.

In [39]:
for run in range(3):
    # define model
    model = Sequential()
    model.add(LSTM(100, activation='tanh', return_sequences=True,  # Tanh activation function
    input_shape=(n_steps, n_features)))
    model.add(LSTM(100, activation='tanh'))
    model.add(Dense(n_features))
    model.compile(optimizer='adam', loss='mse')

    # fit model
    model.fit(X, y, epochs=400, verbose=0)

    # demonstrate prediction
    x_input = array([[70,75,145], [80,85,165], [90,95,185]])
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)

    print(f"Multivariate Stacked LSTM, Tanh activation function, run {run+1}: {yhat}")

Multivariate Stacked LSTM, Tanh activation function, run 1: [[36.7414  39.74856 50.77862]]
Multivariate Stacked LSTM, Tanh activation function, run 2: [[39.710213 39.667255 48.72996 ]]
Multivariate Stacked LSTM, Tanh activation function, run 3: [[39.49917  42.013638 47.429005]]


In [38]:
for run in range(3):
    # define model
    model = Sequential()
    model.add(LSTM(100, activation='sigmoid', return_sequences=True,  # Sigmoid activation function
    input_shape=(n_steps, n_features)))
    model.add(LSTM(100, activation='sigmoid'))
    model.add(Dense(n_features))
    model.compile(optimizer='adam', loss='mse')

    # fit model
    model.fit(X, y, epochs=400, verbose=0)

    # demonstrate prediction
    x_input = array([[70,75,145], [80,85,165], [90,95,185]])
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)

    print(f"Multivariate Stacked LSTM, Sigmoid activation function, run {run+1}: {yhat}")

Multivariate Stacked LSTM, Sigmoid activation function, run 1: [[29.55159 28.58179 33.6874 ]]
Multivariate Stacked LSTM, Sigmoid activation function, run 2: [[25.369205 25.362516 27.13043 ]]
Multivariate Stacked LSTM, Sigmoid activation function, run 3: [[24.846397 22.968704 28.300774]]


### 8.
What are the primary differences between the two LSTM models? Describe and explain what is occuring differently.

In a Multivariate Vanilla LSTM the last of the timeseries data is being predicted based on the input of the other (multiple) timeseries data. In a Multivariate Stacked LSTM all timeseries data is being predicted, which can be referred to as 'multivariate forecasting'.

### 9.
Provide a single table that illustrates the different LSTM models, the activation functions used, and the average of the 3 results (predictions) you received to 3 decimal places. What are the general findings?

| LSTM Model | Activation Function Used | Avg. of 3 Results | General Findings |
| ---------- | ------------------------ | ----------------- | ---------------- |
| MV Vanilla | Relu                     | 210.362           | 2nd most accurate |
| MV Vanilla | Tanh                     | 010.285           | Completely inaccurate |
| MV Vanilla | Sigmoid                  | 006.938           | Completely inaccurate |
| MV Stacked | Relu                     | 100.900, 106.016, 207.019 | 1st most accurate |
| MV Stacked | Tanh                     | 038.65, 040.477, 048.979  | Completely inaccurate |
| MV Stacked | Sigmoid                  | 026.589, 025.638, 29.706  | Completely inaccurate |